# *FTheoryTools*

* Authors: Martin Bies, Mikelis Emils Mikelsons, Andrew P. Turner
* Version: OSCAR version 1.1.0
* Last modified: June 21th, 2024

This tutorial provides a brief introduction to the *FTheoryTools* component of *OSCAR*.

The aim is to describe some basic constructions. In order to facilitate this introduction, we are very selective in what we present. For a full description of the available functionality, see https://docs.oscar-system.org/stable/Experimental/FTheoryTools/introduction/. It is worth mentioning that *FTheoryTools* is still (as of June 21st, 2024) in its experimental stage in *OSCAR*. Consequently, major changes may happen in the future.

For this tutorial, no prior knowledge of *OSCAR* is necessary.

## Content
1. [An SU(5) Tate model from scratch](#1-An-SU(5)-Tate-model-from-scratch)
   - [Construction](#1.A-Construction)
   - [Properties and Attributes](#1.B-Properties-and-Attributes)
2. [Literature Models and Singularity Resolution](#2-Literature-Models-and-Singularity-Resolution)
   - [Creating a Literature Model](#2-A-Creating-a-Literature-Model)
   - [Attributes and Properties of Literature Models](#2.B-Attributes-and-Properties-of-Literature-Models)
   - [Singularity Resolution](#2.C-Singularity-Resolution)
    - [Other Functionality](#2.D-Other-Functionality)
3. [The Quadrillion F-Theory Standard Models](#3-The-Quadrillion-F-Theory-Standard-Models)
    - [Base Geometry](#3.A-Base-Geometry)
    - [The $C_{i}$ Curves](#3.B-The-C_i-Curves)

In [1]:
using Oscar

  ___   ____   ____    _    ____
 / _ \ / ___| / ___|  / \  |  _ \   |  Combining ANTIC, GAP, Polymake, Singular
| | | |\___ \| |     / _ \ | |_) |  |  Type "?Oscar" for more information
| |_| | ___) | |___ / ___ \|  _ <   |  Manual: https://docs.oscar-system.org
 \___/ |____/ \____/_/   \_\_| \_\  |  1.2.0-DEV #master 1c7d9d5 2024-06-20


# 1 An SU(5) Tate model from scratch

## 1.A Construction

We consider an SU(5) Tate model over an arbitrary base. To construct this model, we begin with a $\mathbb{P}^{2, 3, 1}$ fibration over a complex $n$-dimensional base manifold $B$, such that the homogeneous coordinates $[x : y : z]$ transform as
    $$x \in H^0(B, \bar{K}_B^{\otimes 2})\,, \quad y \in H^0(B, \bar{K}_B^{\otimes 3})\,, \quad z \in H^0(B, \mathcal{O}_B)\,,$$
with $K_B$ the canonical bundle of the base. A Tate model is realized as a hypersurface in this $(n + 2)$-dimensional ambient space given by the locus
    $$y^2 + a_1 x y z + a_3 y z^3 = x^3 + a_2 x^2 z^2 + a_4 x z^4 + a_6 z^6\,,$$
with $a_i \in H^0(B, \bar{K}_B^{\otimes i})$. These data define an elliptic fibration $X$ over the base $B$.

For the SU(5) Tate model, we tune the parameters $a_i$ of the model so they become proportional to specific powers of $w$, with $w = 0$ a divisor in the base $B$:
    $$a_1 = w^0 a_{1, 0}\,, \quad a_2 = w^1 a_{2, 1}\,, \quad a_3 = w^2 a_{3, 2}\,, \quad a_4 = w^3 a_{4, 3}\,, \quad a_6 = w^5 a_{6, 5}\,.$$

With FTheoryTools, we can create this model with the following code. First, we create an auxiliary base ring with the parameters needed to define our model:

In [2]:
auxiliary_base_ring, (a10, a21, a32, a43, a65, w) = QQ["a10", "a21", "a32", "a43", "a65", "w"]

(Multivariate polynomial ring in 6 variables over QQ, QQMPolyRingElem[a10, a21, a32, a43, a65, w])

In [3]:
auxiliary_base_grading = [1 2 3 4 6 0; 0 -1 -2 -3 -5 1]

2×6 Matrix{Int64}:
 1   2   3   4   6  0
 0  -1  -2  -3  -5  1

In [4]:
a1 = a10

a10

In [5]:
a2 = a21 * w

a21*w

In [6]:
a3 = a32 * w^2

a32*w^2

In [7]:
a4 = a43 * w^3

a43*w^3

In [8]:
a6 = a65 * w^5

a65*w^5

In [9]:
ais = [a1, a2, a3, a4, a6]

5-element Vector{QQMPolyRingElem}:
 a10
 a21*w
 a32*w^2
 a43*w^3
 a65*w^5

Then, we define the Tate model as given above (in this case, we will choose $d = 3$ for the dimension of the base):

In [10]:
su5_tate = global_tate_model(auxiliary_base_ring, auxiliary_base_grading, 3, ais)

Assuming that the first row of the given grading is the grading under Kbar



Global Tate model over a not fully specified base

## 1.B Properties and Attributes

We can then extract the Tate polynomial, the Tate sections, base spaces etc. of this model.

In [11]:
su5_tate_poly = tate_polynomial(su5_tate)

-a10*x*y*z + a21*w*x^2*z^2 - a32*w^2*y*z^3 + a43*w^3*x*z^4 + a65*w^5*z^6 + x^3 - y^2

In [12]:
base_space(su5_tate)

A family of spaces of dimension d = 3

In [13]:
ambient_space(su5_tate)

A family of spaces of dimension d = 5

This model has singularities, and various information about these singularities determines the physics of the model. First, we compute the discriminant to determine where the model is singular:

In [14]:
su5_disc = factor(discriminant(su5_tate))

(1//16) * w^5 * (a10^6*a65 - a10^5*a32*a43 + a10^4*a21*a32^2 + 12*a10^4*a21*a65*w - a10^4*a43^2*w - 8*a10^3*a21*a32*a43*w - a10^3*a32^3*w - 36*a10^3*a32*a65*w^2 + 8*a10^2*a21^2*a32^2*w + 48*a10^2*a21^2*a65*w^2 - 8*a10^2*a21*a43^2*w^2 + 30*a10^2*a32^2*a43*w^2 - 72*a10^2*a43*a65*w^3 - 16*a10*a21^2*a32*a43*w^2 - 36*a10*a21*a32^3*w^2 - 144*a10*a21*a32*a65*w^3 + 96*a10*a32*a43^2*w^3 + 16*a21^3*a32^2*w^2 + 64*a21^3*a65*w^3 - 16*a21^2*a43^2*w^3 - 72*a21*a32^2*a43*w^3 - 288*a21*a43*a65*w^4 + 27*a32^4*w^3 + 216*a32^2*a65*w^4 + 64*a43^3*w^4 + 432*a65^2*w^5)

(Careful with the above method. It assumes that a family of spaces more or less work as a toric space internally. This is to be discussed and possibly overhauled in the future development process. As a consequence, the output might change. However, the above result is of course correct and as expected in the literature.)

We see that the discriminant vanishes to order 5 at $w = 0$.

We can also compute the coefficients $f$ and $g$ of the corresponding Weierstrass model (defined analogously to our Tate model, but with the hypersurface equation given in short Weierstrass form):

In [15]:
su5_weierstrass = weierstrass_model(su5_tate)

Weierstrass model over a not fully specified base

In [16]:
su5_f = weierstrass_section_f(su5_weierstrass)

-1//48*a10^4 - 1//6*a10^2*a21*w + 1//2*a10*a32*w^2 - 1//3*a21^2*w^2 + a43*w^3

In [17]:
su5_g = weierstrass_section_g(su5_weierstrass)

1//864*a10^6 + 1//72*a10^4*a21*w - 1//24*a10^3*a32*w^2 + 1//18*a10^2*a21^2*w^2 - 1//12*a10^2*a43*w^3 - 1//6*a10*a21*a32*w^3 + 2//27*a21^3*w^3 - 1//3*a21*a43*w^4 + 1//4*a32^2*w^4 + a65*w^5

These sections do not vanish at $w = 0$:

In [18]:
factor(su5_f)

(-1//48) * (a10^4 + 8*a10^2*a21*w - 24*a10*a32*w^2 + 16*a21^2*w^2 - 48*a43*w^3)

In [19]:
factor(su5_g)

(1//864) * (a10^6 + 12*a10^4*a21*w - 36*a10^3*a32*w^2 + 48*a10^2*a21^2*w^2 - 72*a10^2*a43*w^3 - 144*a10*a21*a32*w^3 + 64*a21^3*w^3 - 288*a21*a43*w^4 + 216*a32^2*w^4 + 864*a65*w^5)

This indicates that the singularity at $w = 0$ has Kodaira type $I_5$, which gives rise to an SU(5) gauge group, as desired. This analysis can also be carried out automatically:

In [20]:
singular_loci(su5_tate)

2-element Vector{Tuple{MPolyIdeal{<:MPolyRingElem}, Tuple{Int64, Int64, Int64}, String}}:
 (Ideal with 1 generator, (0, 0, 1), "I_1")
 (Ideal (w), (0, 0, 5), "Split I_5")

This indicates a split (indicating the Tate monodromy type) $I_5$ singularity at $w = 0$, and identifies the rest of the discriminant as a type $I_1$ singularity.

(Note that we are using the Kodaira classification, despite the fact that $X$ is not an elliptic surface; we will further abuse the classification by using it at codimension higher than 1. This is standard in the physics literature, and though acknowledged as improper, always gives consistent physical results in known examples.)

In order to analyze the fibers over the singular locus, we wish to resolve the singularities of the space $X$. However, due to physical constraints (in order to maintain supersymmetry), we restrict ourselves only to crepant resolutions, and we leave any terminal singularities unresolved. The particular model we are considering can be crepantly resolved through codimension 3 by the following sequence of blowups:
    $$X_4 \xrightarrow{(b_{3, 1}, e_2)} X_3 \xrightarrow{(b_{2, 2}, b_{2, 3})} X_2 \xrightarrow{(b_{1, 1}, b_{1, 2}, e_1)} X_1 \xrightarrow{(x, y, w)} X\,,$$
where the center of each blowup is indicated above the corresponding arrow. For blowup $i$ with center $(g_1, \dotsc, g_m)$, the blowup is carried out as $g_j \to e_i b_{i, j}$, so that $e_i = 0$ is the exceptional locus of the blowup.

The blowup above can be carried out using *FTheoryTools*. We turn to this next:

# 2 Literature Models and Singularity Resolution

## 2.A Creating a Literature Model

We begin by creating the above model anew. However, this time we choose a different route:
* The above model is a well-known literature construction, and we provide specialized constructor for such standard constructions. In particular, these constructors will know about known resolutions, such as the one mentioned in the previous paragraph.
* We create this model over a concrete base space. Thereby, we can explicitly execute the resolution in question.

In [21]:
B3 = projective_space(NormalToricVariety, 3)

Normal toric variety

In [22]:
w = torusinvariant_prime_divisors(B3)[1]

Torus-invariant, prime divisor on a normal toric variety

In [23]:
t2 = literature_model(arxiv_id = "1109.3454", equation = "3.1", base_space = B3, defining_classes = Dict("w" => w), completeness_check = false)

Construction over concrete base may lead to singularity enhancement. Consider computing singular_loci. However, this may take time!



Global Tate model over a concrete base -- SU(5)xU(1) restricted Tate model based on arXiv paper 1109.3454 Eq. (3.1)

Note that in the above case, we used the arxiv identifier of the paper in question as well as the equation number in said paper to fully specify the model that we seek to construct. Of course, other information can be provided too and often it is also important to specify the type(Weierstrass, Hypersurface etc.) as multiple forms might be available.

For convenience, a simplified constructor for models is also supported, namely, instead of the meta data of the article, this constructor accepts an integer, which specifies the position of this model in our database. This is useful for commonly used models, whose index could be memorised:

In [24]:
t2 = literature_model(1, base_space = B3, defining_classes = Dict("w" => w), completeness_check = false)

Construction over concrete base may lead to singularity enhancement. Consider computing singular_loci. However, this may take time!



Global Tate model over a concrete base -- SU(5)xU(1) restricted Tate model based on arXiv paper 1109.3454 Eq. (3.1)

## 2.B Attributes and Properties of Literature Models

Of course, the now constructed model knows about its origins in the literature:

In [25]:
arxiv_id(t2)

"1109.3454"

In [26]:
paper_authors(t2)

3-element Vector{String}:
 "Sven Krause"
 "Christoph Mayrhofer"
 "Timo Weigand"

In [27]:
paper_title(t2)

"\$G_4\$ flux, chiral matter and singularity resolution in F-theory compactifications"

In [28]:
model_description(t2)

"SU(5)xU(1) restricted Tate model"

In [29]:
gauge_algebra(t2)

2-element Vector{LinearLieAlgebra{QQBarFieldElem}}:
 Special linear Lie algebra of degree 5 over QQBar
 Linear Lie algebra with 1x1 matrices over QQBar

But of course, we can also just compute "standard" information:

In [30]:
singular_loci(t2)

2-element Vector{Tuple{MPolyIdeal{<:MPolyRingElem}, Tuple{Int64, Int64, Int64}, String}}:
 (Ideal with 1 generator, (0, 0, 1), "I_1")
 (Ideal (x1), (0, 0, 5), "Split I_5")

In [31]:
length(singular_loci(t2))

2

In [32]:
base_space(t2)

Normal, 3-dimensional toric variety without torusfactor

In [33]:
cox_ring(base_space(t2))

Multivariate polynomial ring in 4 variables over QQ graded by
  x1 -> [1]
  x2 -> [1]
  x3 -> [1]
  x4 -> [1]

In [34]:
cox_ring(ambient_space(t2))

Multivariate polynomial ring in 7 variables over QQ graded by
  x1 -> [1 0]
  x2 -> [1 0]
  x3 -> [1 0]
  x4 -> [1 0]
  x -> [8 2]
  y -> [12 3]
  z -> [0 1]

Crucially, the model *t2* has information about resolutions that have been worked out in the literature. In the case at hand:

In [35]:
resolutions(t2)

1-element Vector{Vector{Vector}}:
 [[["x", "y", "w"], ["y", "e1"], ["x", "e4"], ["y", "e2"], ["x", "y"]], ["e1", "e4", "e2", "e3", "s"]]

## 2.C Singularity Resolution

Let us now resolve this model. Given that a sequence of blowups is known, we "just" have to execute these blowups. Note, however, that this step may be technically very challenging, for instance if schemes are employed as bases. Fortunately in the case at hand the resolution is super easy to execute (but still tedious and very time consuming if done by hand).

To resolve this model instantly and without any hard and tedious "human" labor, we just have to select which resolution/blowup sequence the computer should employ. In the case at hand there is only one sequence of (non-weighted) blowups known, as displayed above. Our choice is thus simply "1", which says "resolve with the first available blowup resolution sequence". Here is the result:

In [36]:
t2_resolved = resolve(t2, 1)

Partially resolved global Tate model over a concrete base -- SU(5)xU(1) restricted Tate model based on arXiv paper 1109.3454 Eq. (3.1)

Currently, there are no checks implemented to tell if a model has been fully resolved. Rather, internally the system just remembers if resolution techniques were applied and then informs about this by saying that the model is "partially resolved". However, in the above case, the model is actually fully resolved. (Recall, that this refers to a crepant resolution, and that singularities which cannot be resolved crepantly are still around.)

In [37]:
cox_ring(ambient_space(t2_resolved))

Multivariate polynomial ring in 12 variables over QQ graded by
  x1 -> [1 0 0 0 0 0 0]
  x2 -> [0 1 0 0 0 0 0]
  x3 -> [0 1 0 0 0 0 0]
  x4 -> [0 1 0 0 0 0 0]
  x -> [0 0 1 0 0 0 0]
  y -> [0 0 0 1 0 0 0]
  z -> [0 0 0 0 1 0 0]
  e1 -> [0 0 0 0 0 1 0]
  e4 -> [0 0 0 0 0 0 1]
  e2 -> [-1 -3 -1 1 -1 -1 0]
  e3 -> [0 4 1 -1 1 0 -1]
  s -> [2 6 -1 0 2 1 1]

In [38]:
tate_polynomial(t2_resolved);

For this model we can also perform a non-toric resolution:

In [39]:
w = 2 * torusinvariant_prime_divisors(B3)[1]

Torus-invariant, non-prime divisor on a normal toric variety

In [40]:
t = literature_model(arxiv_id = "1109.3454", equation = "3.1", base_space = B3, defining_classes = Dict("w" => w));

Construction over concrete base may lead to singularity enhancement. Consider computing singular_loci. However, this may take time!



In [41]:
blow_up(t, ["x", "y", string(explicit_model_sections(t)["w"])]; coordinate_name = "e1")

Partially resolved global Tate model over a concrete base -- SU(5)xU(1) restricted Tate model based on arXiv paper 1109.3454 Eq. (3.1)

This is now the starting point of more refined investigations, say of topological data to gauge for instance the Tadpole cancellation condition, or to work out the chiral spectrum. Or to work out the vector-like spectrum, coupling strength of Yukawa interactions etc. We do not yet touch upon this subject. However, we hope to touch on this material once the foundational material has been extended to schemes and families of spaces as base spaces.

## 2.D Other Functionality

It is possible to tune constructed models to change certain attributes, e.g. in an attempt to engineer a larger gauge group. We can also tune a model by fixing a special choice for the model sections.
Note that it is in particular possible to set a section to zero:

In [42]:
Kbar = anticanonical_divisor(B3)

Torus-invariant, non-prime divisor on a normal toric variety

In [43]:
F1 = literature_model(arxiv_id = "1408.4808", equation = "3.4", type = "hypersurface", base_space = B3, defining_classes = Dict("s7" => Kbar, "s9" => Kbar), completeness_check = false)

Construction over concrete base may lead to singularity enhancement. Consider computing singular_loci. However, this may take time!



Hypersurface model over a concrete base

In [44]:
hypersurface_equation_parametrization(F1)

s1*u^3 + s2*u^2*v + s3*u*v^2 + s4*v^3 + s5*u^2*w + s6*u*v*w + s7*v^2*w + s8*u*w^2 + s9*v*w^2 + s10*w^3

In [45]:
x1, x2, x3, x4 = gens(cox_ring(base_space(F1)));;

In [46]:
new_choice = Dict("s10" => zero(parent(x1)))

Dict{String, MPolyDecRingElem{QQFieldElem, QQMPolyRingElem}} with 1 entry:
  "s10" => 0

In [47]:
tuned_F1 = tune(F1, new_choice)

Hypersurface model over a concrete base

In [48]:
is_zero(explicit_model_sections(tuned_F1)["s10"])

true

In [49]:
explicit_model_sections(tuned_F1)

Dict{String, MPolyDecRingElem{QQFieldElem, QQMPolyRingElem}} with 10 entries:
  "s1"  => 8791199083695932474*x1^4 + 7643479792612114091*x1^3*x2 - 32044866152…
  "s8"  => -7213043686742499701*x1^4 - 3807201613773135548*x1^3*x2 + 6319967702…
  "s5"  => 33927466523590420*x1^4 - 224514977524182041*x1^3*x2 + 19910779382562…
  "s6"  => -3807450074396734867*x1^4 - 8436982387215131363*x1^3*x2 - 8779812591…
  "s7"  => 8688563996494228138*x1^4 - 1073869156965394580*x1^3*x2 - 36435089363…
  "s2"  => 5908469472187553623*x1^4 - 7903242100205887616*x1^3*x2 + 28943427036…
  "s4"  => -3252539725390639171*x1^4 - 2660939820414109260*x1^3*x2 - 2723212825…
  "s9"  => -4437140836833732292*x1^4 + 5560335080038301206*x1^3*x2 - 5922897403…
  "s3"  => -7505763851429914082*x1^4 - 5177260832765410957*x1^3*x2 - 4805410458…
  "s10" => 0

Lastly, all of the models and information that is stored about the models functions as a database, that will be expanded with time:

In [50]:
display_all_literature_models()

Model 1:
Dict{String, Any}("journal_section" => "3", "arxiv_page" => "10", "arxiv_id" => "1109.3454", "arxiv_version" => "2", "journal_equation" => "3.1", "journal_page" => "9", "arxiv_equation" => "3.1", "journal_doi" => "10.1016/j.nuclphysb.2011.12.013", "arxiv_section" => "3", "journal" => "Nucl. Phys. B", "file" => "model1109_3454.json", "arxiv_doi" => "10.48550/arXiv.1109.3454", "model_index" => "1", "type" => "tate")

Model 2:
Dict{String, Any}("journal_section" => "B", "arxiv_page" => "32", "arxiv_id" => "1208.2695", "arxiv_version" => "2", "journal_equation" => "B.5", "journal_page" => "33", "arxiv_equation" => "B.5", "journal_doi" => "10.1007/JHEP10(2012)128", "arxiv_section" => "B", "journal" => "JHEP", "file" => "model1208_2695-1.json", "arxiv_doi" => "10.48550/arXiv.1208.2695", "model_index" => "2", "type" => "hypersurface")

Model 3:
Dict{String, Any}("journal_section" => "B", "arxiv_page" => "34", "arxiv_id" => "1208.2695", "arxiv_version" => "2", "journal_equation" => "B

# 3 The Quadrillion F-Theory Standard Models

## 3.A Base Geometry

A yet more special instance of literature models are the Quadrillion F-theory Standard Models
(F-theory QSMs) [CHLLT19](@cite). Those hypersurface models come in 708 different families.

The base geometry of an F-theory QSM is obtained from triangulating one of 708 reflexive 3-dimensional
polytopes. The models, whose bases are obtained from triangulations of the same polytope form a family.
The following information on the polytope in question and its triangulations is available within the database:

In [51]:
qsm_model = literature_model(arxiv_id = "1903.00009", model_parameters = Dict("k" => 4))

Hypersurface model over a concrete base

The following method returns the vertices of the polytope the the base of the F-Theory QSM is
build from. Note that those vertices are normalized according to the Polymake standard
to rational numbers:

In [52]:
vertices(qsm_model)

4-element Vector{Vector{QQFieldElem}}:
 [-1, -1, -1]
 [1, -1//2, -1//2]
 [-1, 2, -1]
 [-1, -1, 5]

Of the 3-dimensional reflexive polytope that the base of this F-theory model is build from,
the following method returns the index within the Kreuzer-Skarke list:

In [53]:
polytope_index(qsm_model)

4

For a 3-dimensional reflexive polytope in the Kreuzer-Skarke list, the list of
full (sometimes also called fine), regular, star triangulations can be extremely
large. Consequently, one may wonder if the triangulations can be enumerated in a
somewhat reasonable time (say 5 minutes on a personal computer). The following method tries
to provide an answer to this. It returns `true` if one should expect a timly response
to the atttempt to enumerate all (full, regular, star) triangulations. Otherwise, this
method returns `false`:

In [54]:
has_quick_triangulation(qsm_model)

true

In order to enumerate the number of full, regular, star triangulations of a
3-dimensional reflexive polytope, it is possible to first find the corresponding
triangulations of all facets of the polytope [HT17](@cite). A first indication for
the complexity of this triangulation task is the maximum number of lattice points
in a facet of the polytope in question. The following method returns this maximal number of
lattice points:

In [55]:
max_lattice_pts_in_facet(qsm_model)

16

The following method returns an estimate for the number of full, regular, star triangulations
of the 3-dimensional reflexive polytope, those triangulations define the possible base
spaces of the F-theory QSM in question:

In [56]:
estimated_number_of_triangulations(qsm_model)

212533333333

Beyond the polytope and its triangulations, a number of other integers are of key importance, for example, Hodge numbers are also supported in the database.

## 3.B The $C_{i}$ Curves

More recently, a research program estimated the exact massless spectra of the F-theory QSMs
(cf. [Bie24](@cite)). These studies require yet more information about the F-theory QSM geometries,
which are supported by the database.

First, recall that (currently), the base of an F-theory QSM is a 3-dimensional toric variety $B3$.
Let $s \in H^0(B3, \bar{K}_{B3}^{\otimes i})$, then $V(s)$ is a K3-surface. Moreover, let $x_{i}$ be the coordinates
of the Cox ring of $B3$. Then $V(x_{i})$ is a divisor in $B3$. Consequently, $C_{i} = V(x_{i}) \cap V(s)$
is a divisor in the K3-surface $V(s)$. For the root bundle counting program, these curves $C_{i}$ are
of ample importance (cf. [Bie24](@cite)). The following information on these curves is supported:

The following method returns the genera of the $C_{i}$ curves.
Recall that $C_{i} = V(x_{i}, s)$, where $x_{i}$ is a homogeneous
coordinate of the 3-dimensional toric base space $B3$ of the
QSM hypersurface model in question, and $s$ is a generic
section of the anticanonical bundle of $B3$. Consequently,
we may use the coordinates $x_{i}$ as labels for the curves $C_{i}$:

In [57]:
my_key = collect(keys(degrees_of_kbar_restrictions_to_ci_curves(qsm_model)))[1]

x7

In [59]:
genera_of_ci_curves(qsm_model)[my_key]

0

The anticanonical divisor of the 3-dimensional toric base space $B3$ of the
QSM hypersurface model in question can be restricted to the $C_{i}$ curves. The
result of this operation is a line bundle. The following method returns the degree of
this line bundle for every $C_{i}$ curve:

In [58]:
degrees_of_kbar_restrictions_to_ci_curves(qsm_model)[my_key]

0

The topological intersection numbers among $C_{i}$ curves are also of ample importance.
The following method returns those intersection numbers in the form of a matrix:

In [60]:
topological_intersection_numbers_among_ci_curves(qsm_model)

29×29 Matrix{Int64}:
 -2   0   0   0   1   0   0   0   0   1  …  0  0   0  0  0   0   0  0   0   0
  0  -2   0   0   0   0   0   0   0   0     0  0   0  0  0   0   1  0   1   2
  0   0  -2   0   0   0   0   0   0   0     0  0   0  0  0   2   0  0   0   0
  0   0   0  -2   0   0   0   0   1   0     0  0   1  0  0   0   0  0   0   0
  1   0   0   0  -2   1   0   0   0   0     0  0   0  0  0   0   0  0   0   0
  0   0   0   0   1  -2   1   0   0   0  …  0  0   0  0  0   0   0  0   0   0
  0   0   0   0   0   1  -2   1   0   0     0  0   0  0  0   0   0  0   0   0
  0   0   0   0   0   0   1  -2   1   0     0  0   0  0  0   0   0  0   0   0
  0   0   0   1   0   0   0   1  -2   0     0  0   0  0  0   0   0  0   0   0
  1   0   0   0   0   0   0   0   0  -2     0  0   0  0  0   0   0  0   0   0
  0   0   0   0   0   0   0   0   0   0  …  0  0   0  0  0   0   0  0   0   0
  0   0   0   0   0   0   0   0   0   0     0  0   0  0  0   0   0  0   0   0
  0   0   0   0   0   0   0   0   0   0    

Some of the $C_{i}$ curves are trivial, in that $V(x_{i}, s)$ is the empty set.
The following method returns the vector of all indices of trivial $C_{i}$ curves.
That is, should $V(x_{23}, s)$ be the empty set, then 23 will be included in
the returned list:

In [61]:
indices_of_trivial_ci_curves(qsm_model)

10-element Vector{Int64}:
 23
 22
 18
 19
 20
 26
 10
 11
 12
 15

The topological intersection numbers among the non-trivial $C_{i}$ curves are used
frequently. The following method returns those intersection numbers in the form of a matrix:

In [62]:
topological_intersection_numbers_among_nontrivial_ci_curves(qsm_model)

19×19 Matrix{Int64}:
 -2   0   0   0   1   0   0   0   0   1   0   0   0   1   0   0   0   0   0
  0  -2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   1   2
  0   0  -2   0   0   0   0   0   0   0   0   1   1   0   0   2   0   0   0
  0   0   0  -2   0   0   0   0   1   0   1   0   0   0   1   0   0   0   0
  1   0   0   0  -2   1   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   1  -2   1   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   1  -2   1   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   1  -2   1   0   0   0   0   0   0   0   0   0   0
  0   0   0   1   0   0   0   1  -2   0   0   0   0   0   0   0   0   0   0
  1   0   0   0   0   0   0   0   0  -2   0   1   0   0   0   0   0   0   0
  0   0   0   1   0   0   0   0   0   0  -2   0   1   0   0   0   0   0   0
  0   0   1   0   0   0   0   0   0   1   0  -2   0   0   0   0   0   0   0
  0   0   1   0   0   0   0   0   0   0   1   0  -2   0   0   0   0

The collection of the $C_{i}$ curves form a nodal curve. To every nodal curve one can associate a
(dual) graph. In this graph, every irreducible component of the nodal curve becomes a node/vertex
of the dual graph, and every nodal singularity of the nodal curve turns into an edge of the dual
graph. In the case at hand, this is rather simple.

The $C_{i}$ curves turn into the irreducible components of the nodal curve. Certainly, we only need
to focus on the non-trivial $C_{i}$ curves. A non-trivial $C_{i}$ curve can split into multiple irreducible
components. This is taken into acccount when the nodes/vertices of the dual graph are constructed.

The topological intersection numbers among the $C_{i}$ curves (or rather, their irreducible components)
tells us how many nodal singularities link the $C_{i}$ curves (or rather, their irreducible components)
in question. Hence, if the topological intersection numbers is zero, there is no edge between the
corresponding nodes. Otherwise, if the topological intersection number is positive, say $n$, then
there are exactly $n$ edges between the nodes in question.

The following method returns the dual graph of the QSM model in question.
Note that no labels are (currently) attached to the vertices/nodes or edges.
To understand/read this graph correctly, one of the following methods listed below:

In [63]:
dual_graph(qsm_model)

Undirected graph with 21 nodes and the following edges:
(5, 1)(6, 5)(7, 6)(8, 7)(9, 4)(9, 8)(10, 1)(11, 4)(12, 3)(12, 10)(13, 3)(13, 11)(14, 1)(15, 4)(16, 3)(17, 3)(18, 2)(18, 14)(19, 2)(19, 15)(20, 2)(20, 16)(21, 2)(21, 17)

The following method returns a vector with labels for each node/vertex of the dual graph of the QSM
model in question. Those labels allow to understand the geometric origin of the node/vertex.

Specifically, recall that those nodes are associated to the $C_{i}$ curves, which are in turn
given by $C_{i} = V(x_{i}, s)$. $x_{i}$ is a homogenous coordinate of the 3-dimensional toric base space
B3 of the QSM in question, and s is a generic section of the anticanonical bundle of $B3$.

Only non-trivial $C_{i}$ curves correspond to vertices/nodes of the dual graph.

If $C_{i}$ is irreducible and corresponds to the k-th component, then the label "Ci"
appears at position k of the vector returned by this method. However, if $C_{i}$ is
reducible, then we introduce the labels Ci-0, Ci-1, Ci-2 etc. for those irreducible
components of $C_{i}$. If Ci-0 corresponds to the k-th components of the dual graph,
then the label "Ci-0" appears at position k of the vector returned by this method:


In [64]:
components_of_dual_graph(qsm_model)

21-element Vector{String}:
 "C0"
 "C1"
 "C2"
 "C3"
 "C4"
 "C5"
 "C6"
 "C7"
 "C8"
 "C9"
 "C13"
 "C14"
 "C16"
 "C17"
 "C21"
 "C24-0"
 "C24-1"
 "C25"
 "C27"
 "C28-0"
 "C28-1"

The anticanonical bundle of the toric 3-dimensional base space of the F-theory QSM in
question can be restricted to the (geometric counterparts of the) nodes/vertices of
the dual graph. The result is a line bundle for each node/vertex. The following method returns
a vector with the degrees of these line bundles:

In [65]:
degrees_of_kbar_restrictions_to_components_of_dual_graph(qsm_model)["C28-1"]

0

The following method returns a vector with the genera of the (geometric
counterparts of the) nodes/vertices of the dual graph:

In [66]:
genera_of_components_of_dual_graph(qsm_model)["C28-1"]

0

The dual graph is essential in counting root bundles (cf. [BCL21](@cite)). It turns out, that one
can simplify this graph so that the computations at hand can be conducted on a simpler graph
instead. The following functionality exists to access this simplified dual graph:
```@docs
simplified_dual_graph(m::AbstractFTheoryModel)
components_of_simplified_dual_graph(m::AbstractFTheoryModel)
degrees_of_kbar_restrictions_to_components_of_simplified_dual_graph(m::AbstractFTheoryModel)
genera_of_components_of_simplified_dual_graph(m::AbstractFTheoryModel)
```